# 🏛️ Microsoft Access VBA & Automation Tutorial

This tutorial provides a full, hands-on introduction to **VBA programming in Microsoft Access**. It explains how to automate forms, tables, queries, and reports; how to use **DAO** and **ADO** for database access; and how to integrate Access with **Excel**, **Outlook**, and other Office applications.

## 📘 Overview

Access VBA extends your database beyond macros. You can:
- Automate complex queries and reports
- Build data-driven workflows (ETL)
- Generate Excel and PDF reports automatically
- Control Outlook and Word for notifications and documents

### Access Object Model (AOM)
```
Application
│
├── CurrentDb (DAO.Database)
│    ├── TableDefs
│    ├── QueryDefs
│    ├── Recordsets
│    └── Relations
│
├── Forms
│    └── Controls
├── Reports
└── DoCmd
     ├── OpenForm / OpenReport
     ├── RunSQL / TransferSpreadsheet
     └── OutputTo / SendObject
```

## ⚙️ Working in the VBA Environment

- Open the Visual Basic Editor with `Alt + F11`.
- Use **modules** for code and **forms** for event-based logic.
- Common Access objects:
  - `CurrentDb` → DAO database object for the current database
  - `Forms!FormName` → active form instance
  - `Reports!ReportName` → open report
  - `DoCmd` → execute Access commands
  - `Application` → top-level Access object

In [ ]:
Sub PrintFormData()
    Debug.Print Forms!frmEmployees!txtEmployeeName
End Sub

## 🧩 DAO Fundamentals

DAO (Data Access Objects) gives high-performance local access to tables and queries using the Jet/ACE engine.

In [ ]:
Sub DAOExample()
    Dim db As DAO.Database
    Dim rs As DAO.Recordset
    Set db = CurrentDb
    Set rs = db.OpenRecordset("SELECT * FROM Employees", dbOpenDynaset)
    
    Do Until rs.EOF
        Debug.Print rs!EmployeeName, rs!Department
        rs.MoveNext
    Loop
    rs.Close
End Sub

### Creating a New Table with DAO

In [ ]:
Sub CreateDeptTable()
    Dim db As DAO.Database
    Dim tdf As DAO.TableDef

    Set db = CurrentDb
    Set tdf = db.CreateTableDef("Departments")
    tdf.Fields.Append tdf.CreateField("DeptID", dbLong)
    tdf.Fields.Append tdf.CreateField("DeptName", dbText, 50)
    db.TableDefs.Append tdf
End Sub

## 🧮 Editing and Updating Data

In [ ]:
Sub UpdateSalaries()
    CurrentDb.Execute "UPDATE Employees SET Salary = Salary * 1.05 WHERE Dept='Finance';", dbFailOnError
    MsgBox "Salaries updated."
End Sub

## ⚙️ ADO (ActiveX Data Objects)

ADO allows Access to connect to **external databases** such as SQL Server, Oracle, or Excel.

In [ ]:
Sub ConnectExternalADO()
    Dim cn As Object, rs As Object
    Set cn = CreateObject("ADODB.Connection")
    cn.Open "Provider=SQLOLEDB;Data Source=Server01;Initial Catalog=Finance;Integrated Security=SSPI;"

    Set rs = CreateObject("ADODB.Recordset")
    rs.Open "SELECT TOP 10 * FROM Budget", cn, 1, 1

    Do Until rs.EOF
        Debug.Print rs!Dept, rs!Amount
        rs.MoveNext
    Loop
    rs.Close: cn.Close
End Sub

## 📑 QueryDefs and Parameters

In [ ]:
Sub QueryWithParameter()
    Dim qdf As DAO.QueryDef, rs As DAO.Recordset
    Set qdf = CurrentDb.CreateQueryDef("", _
        "SELECT * FROM Employees WHERE HireDate > [startDate]")

    qdf!startDate = #1/1/2020#
    Set rs = qdf.OpenRecordset
    Do Until rs.EOF
        Debug.Print rs!EmployeeName
        rs.MoveNext
    Loop
End Sub

## 🧱 Forms, Controls, and Events

Forms contain control-specific code for handling events like `AfterUpdate` or `BeforeUpdate`.

In [ ]:
Private Sub cboDept_AfterUpdate()
    Me.txtMgr = DLookup("Manager", "Departments", "DeptName='" & Me.cboDept & "'")
End Sub

In [ ]:
Private Sub Form_BeforeUpdate(Cancel As Integer)
    If IsNull(Me.EmployeeName) Then
        MsgBox "Name required!", vbExclamation
        Cancel = True
    End If
End Sub

## 🧩 Using `DoCmd` for Automation

`DoCmd` executes Access commands programmatically.

In [ ]:
DoCmd.OpenForm "frmEmployees"
DoCmd.OpenReport "rptSummary", acViewPreview
DoCmd.TransferSpreadsheet acExport, acSpreadsheetTypeExcel12, "qryData", "C:\Data\out.xlsx", True
DoCmd.OutputTo acOutputReport, "rptSummary", acFormatPDF, "C:\Report.pdf"

## 🔗 Access ↔ Excel Automation

In [ ]:
Sub ExportQueryToExcel()
    Dim filePath As String
    filePath = "C:\Exports\Summary_" & Format(Date, "yyyymmdd") & ".xlsx"

    DoCmd.TransferSpreadsheet acExport, acSpreadsheetTypeExcel12Xml, _
        "qryMonthlySummary", filePath, True

    MsgBox "Export complete: " & filePath
End Sub

In [ ]:
Sub ImportExcelData()
    Dim srcPath As String
    srcPath = "C:\Imports\EmployeeData.xlsx"

    DoCmd.TransferSpreadsheet acImport, acSpreadsheetTypeExcel12Xml, _
        "tblEmployees", srcPath, True

    MsgBox "Import complete."
End Sub

In [ ]:
Sub BuildWorkbookFromAccess()
    On Error GoTo ErrHandler
    Dim xlApp As Object, xlWB As Object, rs As DAO.Recordset
    Dim i As Long, r As Long

    Set xlApp = CreateObject("Excel.Application")
    Set xlWB = xlApp.Workbooks.Add
    xlApp.Visible = True
    Set rs = CurrentDb.OpenRecordset("SELECT * FROM qryMonthlySummary")

    For i = 0 To rs.Fields.Count - 1
        xlWB.Sheets(1).Cells(1, i + 1).Value = rs.Fields(i).Name
    Next i

    r = 2
    Do Until rs.EOF
        For i = 0 To rs.Fields.Count - 1
            xlWB.Sheets(1).Cells(r, i + 1).Value = rs.Fields(i).Value
        Next i
        rs.MoveNext: r = r + 1
    Loop

    xlWB.SaveAs "C:\Exports\MonthlyData_" & Format(Date, "yyyymmdd") & ".xlsx"

CleanExit:
    On Error Resume Next
    rs.Close
    Set rs = Nothing: Set xlWB = Nothing: Set xlApp = Nothing
    Exit Sub
ErrHandler:
    MsgBox Err.Description
    Resume CleanExit
End Sub

## ✉️ Access ↔ Outlook Automation

In [ ]:
Sub EmailReportAsPDF()
    Dim rpt As String, outFile As String
    rpt = "rptMonthlyBudget"
    outFile = "C:\Reports\" & rpt & "_" & Format(Date, "yyyymmdd") & ".pdf"

    DoCmd.OutputTo acOutputReport, rpt, acFormatPDF, outFile

    Dim olApp As Object, mail As Object
    Set olApp = CreateObject("Outlook.Application")
    Set mail = olApp.CreateItem(0)
    With mail
        .To = "finance@agency.gov"
        .Subject = "Monthly Budget Report"
        .Body = "Attached is the latest report."
        .Attachments.Add outFile
        .Display
    End With
End Sub

In [ ]:
Sub SendEmailsFromList()
    Dim olApp As Object, mail As Object, rs As DAO.Recordset
    Set olApp = CreateObject("Outlook.Application")
    Set rs = CurrentDb.OpenRecordset("SELECT * FROM tblNotifications WHERE Sent=False")

    Do Until rs.EOF
        Set mail = olApp.CreateItem(0)
        With mail
            .To = rs!Email
            .Subject = "Notification"
            .Body = "Dear " & rs!Name & "," & vbCrLf & rs!Message
            .Display
        End With
        rs.Edit: rs!Sent = True: rs.Update
        rs.MoveNext
    Loop
    rs.Close
End Sub

## 🧠 Full Automation Example – Department Report Distribution

In [ ]:
Sub DistributeFinanceReports()
    On Error GoTo ErrHandler
    Dim rs As DAO.Recordset, olApp As Object, mail As Object
    Dim rpt As String, pdfPath As String
    rpt = "rptDepartmentSummary"

    Set olApp = CreateObject("Outlook.Application")
    Set rs = CurrentDb.OpenRecordset("SELECT Dept, Email FROM tblDepartments")

    Do Until rs.EOF
        pdfPath = "C:\Reports\" & rpt & "_" & rs!Dept & ".pdf"
        DoCmd.OpenReport rpt, acViewPreview, , "Dept='" & rs!Dept & "'"
        DoCmd.OutputTo acOutputReport, rpt, acFormatPDF, pdfPath
        DoCmd.Close acReport, rpt

        Set mail = olApp.CreateItem(0)
        With mail
            .To = rs!Email
            .Subject = "Department Budget Summary – " & rs!Dept
            .Body = "Attached is your department's latest summary."
            .Attachments.Add pdfPath
            .Send
        End With
        rs.MoveNext
    Loop

CleanExit:
    On Error Resume Next
    rs.Close
    Set rs = Nothing: Set mail = Nothing: Set olApp = Nothing
    Exit Sub
ErrHandler:
    MsgBox "Error: " & Err.Description
    Resume CleanExit
End Sub

## 💡 Best Practices

- Use **DAO** for local Access data, **ADO** for external sources.
- Prefer saved queries to embedded SQL.
- Always `.Close` Recordsets and COM objects.
- Release Excel/Outlook objects in reverse creation order.
- Log exports, emails, and errors to an Access table.
- Use configuration tables for dynamic file paths.
- Split Access databases into front-end (UI) and back-end (data).

## 🧾 Summary

- Access VBA automates tables, queries, and reports through **DAO** and **DoCmd**.
- **ADO** connects Access to external systems.
- **Excel automation** enables formatted data exports.
- **Outlook integration** sends reports automatically.
- Combine them into a complete ETL + reporting workflow.